**Algoritmos para Big Data**

**2022/23**

# Exploratory Data Analysis (EDA)
In this lecture, we will carry out some common data analysis to extract insights from data stored in a DataFrame. The main goal is to consolidate knownledge from previous lectures about useful Spark's transformations and actions functions.

# Problem formulation

This exercise is about EDA related to **Fire Department calls for service** in San Francisco, USA. 

We ask you write down a Spark program that:

a) Reads a file containing the dataset under analysis.

b) Provides answers to the following questions about the data.

1. How many distinct types of calls were made to the Fire Department?
2. What are distinct types of calls that were made to the Fire Department?
3. Find out all responses or delayed times that were greater than 5 minutes?
4. What were the most common call types, listed in descending order by count?
5. What zip codes accounted for most common calls and what type were they?
6. What neighbourhoods are in the two top zip codes from the listing in the previous question?

The dataset of concern can also be found via the link 

https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3

# Initial settings

Prior to any computation, let us set some imports and useful functions

## Additional packages and imports

In [ ]:
# If we need to install some packages, e.g. matplotlib

! pip3 install matplotlib seaborn pandas numpy

In [ ]:
# Some general imports 

import os 

# import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Spark's imports that we need hereafter. 
# It can be imported here or further down as we need them, step by step

import sys

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

## Useful visualization functions

Some functions that we can use to plot data but as Python DataFrames.

**Disclaimer**: these functions are broadly distributed among users. Further adjustments are needed and/or advisable. 

We encourage you to use your own plotting functions. 

Remember: *"A picture is worth a thousand words"*

In [ ]:
def plot(df, xcol, ycol):
    sns.lineplot(data=df, x=xcol, y=ycol)

In [ ]:
def plotHistogram(df, xcol, huecol):
    sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")

In [ ]:
def plotScatter(df, xcol, ycol, huecol):
    sns.scatterplot(data=df, x=xcol, y=ycol, hue=huecol)

In [ ]:
def plotScatterMatrix(df, huecol):
    sns.pairplot(data=df, hue=huecol)

In [ ]:
def plotCorrelationMatrix_1(df):
    # compute the correlation matrix
    corr = df.corr()

    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
def plotCorrelationMatrix_2(df):
    # compute a correlation matrix and convert to long-form
    corr_mat = df.corr().stack().reset_index(name="correlation")
    # draw each cell as a scatter point with varying size and color
    g = sns.relplot(
        data=corr_mat,
        x="level_0", y="level_1", hue="correlation", size="correlation",
        palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),
    )

    # tweak the figure to finalize
    g.set(xlabel="", ylabel="", aspect="equal")
    g.despine(left=True, bottom=True)
    g.ax.margins(.02)
    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

# Data ingestion

In [ ]:
# Check working directory

! pwd
! ls -la

In [ ]:
! ls -la fire-department-calls

In [ ]:
# Check file directly using commands head and tail

! head -n 2 fire-department-calls/Fire_Department_Calls_for_Service_Excel_EU.csv
! tail -n 2 fire-department-calls/Fire_Department_Calls_for_Service_Excel_EU.csv

In [ ]:
# Build a SparkSession instance if one does not exist. Notice that we can only have one per JVM
#
# Notice however that, when using a PySpark kernel, a default SparkSession has been created with 
# the name spark. But if we want to use different settings, then we have to create our own.

spark = SparkSession\
    .builder\
    .appName("FireDepartmentCalls")\
    .config("spark.sql.shuffle.partitions",6)\
    .getOrCreate()

## Reading the dataset

In [ ]:
# As the file is quite big, (recall that inferring the schema is expensive for large files) 
# and we know it, so let us use it (How do we know it?)

fire_schema = StructType([StructField('Call Number', IntegerType(), True),
                     StructField('Unit ID', StringType(), True),
                     StructField('Incident Number', IntegerType(), True),
                     StructField('Call Type', StringType(), True),                  
                     StructField('Call Date', StringType(), True),      
                     StructField('Watch Date', StringType(), True),                
                     StructField('Received DtTm', StringType(), True),
                     StructField('Entry DtTm', StringType(), True),
                     StructField('Dispatch DtTm', StringType(), True),
                     StructField('Response DtTm', StringType(), True),
                     StructField('On Scene DtTm', StringType(), True),
                     StructField('Transport DtTm', StringType(), True),
                     StructField('Hospital DtTm', StringType(), True),
                     StructField('Call Final Disposition', StringType(), True),
                     StructField('Available DtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode of Incident', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('Station Area', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('Original Priority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('Final Priority', IntegerType(), True),       
                     StructField('ALS Unit', BooleanType(), True),
                     StructField('Call Type Group', StringType(), True),
                     StructField('Number of Alarms', IntegerType(), True),
                     StructField('Unit Type', StringType(), True),
                     StructField('Unit sequence in call dispatch', IntegerType(), True),
                     StructField('Fire Prevention District', StringType(), True),
                     StructField('Supervisor District', StringType(), True),
                     StructField('Neighborhooods - Analysis Boundaries', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('case_location', StringType(), True),
                     StructField('Analysis Neighborhoods', IntegerType(), True)])
                          

In [ ]:
# Reading the dataset

filename = "fire-department-calls/Fire_Department_Calls_for_Service_Excel_EU.csv"
#fire_df = spark.read.csv(filename, header=True, schema=fire_schema, sep=';')
fire_df = spark.read.csv(filename, header=True, inferSchema="true", sep=';')

## Checking data
Schema, show, count, and statistical information.

In [ ]:
# Schema

fire_df.

In [ ]:
# Show

fire_df.

In [ ]:
# Count

fire_df.

In [ ]:
# Statistical information using describe()
# PS: Output takes time to compute given the number of columns 
#     and size of dataset

for cl in fire_df.columns:
    fire_df.

# Exploratory data analysis

Prior to any further analysis, we should consider upating the data types of time related fields.
The data dictionary provided shows the fields that are considered as of *Date & Time*.

The default format should be MM-dd-yyyy HH:mm:ss.SSS ... 

but it looks like we have MM/dd/yyyy HH:mm:ss PM (or AM)


|Column     | Type| Description |
|:---:|:---:| :---:| 
| **Call Date** |Date & Time| Date the call is received at the 911 Dispatch Center. Used for reporting purposes.|
| **Watch Date** |Date & Time| Watch date when the call is received. Watch date starts at 0800 each morning and ends at 0800 the next day.|
| **Received DtTm** |Date & Time| Date and time of call is received at the 911 Dispatch Center.|
| **Entry DtTm** |Date & Time| Date and time the 911 operator submits the entry of the initical call information into the CAD system.|
| **Dispatch DtTm** |Date & Time| Date and time the 911 operator dispatches this unit to the call.|
| **Response DtTm** |Date & Time| Date and time this unit acknowledges the dispatch and records that the unit is en route to the location of the call.|
| **On Scene DtTm** |Date & Time| Date and time the unit records arriving to the location of the incident.|
| **Transport DtTm** |Date & Time| If this unit is an ambulance, date and time the unit begins the transport unit arrives to hospital.|
| **Hospital DtTm** |Date & Time| If this unit is an ambulance, date and time the unit arrives to the hospital.|
| **Available DtTm** |Date & Time| Date and time this unit is not longer assigned to this call and it is available for another dispatch.|


In [ ]:
# See https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

new_fire_df = ( fire_df
                .withColumn("Call Date ts", F.to_timestamp("Call Date","MM/dd/yyyy"))
                .withColumn("Watch Date ts", F.to_timestamp("Watch Date","MM/dd/yyyy"))
                .withColumn("Received DtTm ts", F.to_timestamp("Received DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Entry DtTm ts", F.to_timestamp("Entry DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Dispatch DtTm ts", F.to_timestamp("Dispatch DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Response DtTm ts", F.to_timestamp("Response DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("On Scene DtTm ts", F.to_timestamp("On Scene DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Transport DtTm ts", F.to_timestamp("Transport DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Entry DtTm ts", F.to_timestamp("Entry DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Hospital DtTm ts", F.to_timestamp("Hospital DtTm","MM/dd/yyyy KK:mm:ss a"))
                .withColumn("Available DtTm ts", F.to_timestamp("Available DtTm","MM/dd/yyyy KK:mm:ss a"))
                )

In [ ]:
# Check changes

new_fire_df.
# or new_fire_df.columns
new_fire_df.

In [ ]:
# Delete old dataframe if no longer needed

del fire_df

In [ ]:
# Cache the DataFrame since we will be performing many operations on it.
# It makes operations faster at expenses of memory storage. 
# Or better not using it!

# new_fire_df.cache()

## Questions to be anwsered

In [ ]:
# Just to review columns' name

new_fire_df.columns

In [ ]:
# Set a short list of main columns just for showing purposes, if needed

main_cols = ['Call Number',
 'Unit ID',
 'Incident Number',
 'Call Type',
 'Call Date',
 'Watch Date',
 'Received DtTm',
 'Entry DtTm',
 'Dispatch DtTm',
 'Response DtTm',
 'On Scene DtTm',
 'Transport DtTm',
 'Hospital DtTm',
 'Call Final Disposition',
 'Available DtTm',
 'Address',
 'City',
 'Zipcode of Incident',
 'Call Date ts',
 'Watch Date ts',
 'Received DtTm ts',
 'Entry DtTm ts',
 'Dispatch DtTm ts',
 'Response DtTm ts',
 'On Scene DtTm ts',
 'Transport DtTm ts',
 'Hospital DtTm ts',
 'Available DtTm ts']

**1) How many distinct types of calls were made to the Fire Department?**

Of course, we will not count "null" strings in that column.

In [ ]:
new_fire_df.

**2) What are the distinct types of calls that were made to the Fire Department?**

In [ ]:
new_fire_df.

**3) Find out all responses or delayed times that were greater than 5 minutes?**

*(from the moment call is received till response is acknowledged and unit is on route)*

1. Creates a new field *Response Delay* with the delay in minutes
2. Filter out the records with delay higher than 5 minutes.

In [ ]:
new_fire_df = new_fire_df.

In [ ]:
cols_to_show = main_cols + ['Response Delay']
new_fire_df.

In [ ]:
( new_fire_df
     .
 
)

**4) What were the most common call types, listed in descending order by count?**

In [ ]:
( new_fire_df
     .
 
)

**5) What zip codes accounted for most common calls and what type were they?**

1. Filter out by Call Type
2. Group them by Call Type and Zipcode of Incident
3. Count them and display in descending order

In [ ]:
( new_fire_df
     .
 
)

**6) What neighbourhoods are in the two top zip codes from the listing in the previous question?**

Probably these two zip codes are somehow related to contested neighbourhood with high reported crimes.

In [ ]:
( new_fire_df
     .
 
)

## Answers regarding the questions above

**1) How many distinct types of calls were made to the Fire Department?**

Of course, we will not count "null" strings in that column.

**2) What are the distinct types of calls that were made to the Fire Department?**

**3) Find out all responses or delayed times that were greater than 5 minutes?**

*(from the moment call is received till response is acknowledged and unit is on route)*

1. Creates a new field *Response Delay* with the delay in minutes
2. Filter out the records with delay higher than 5 minutes.

**4) What were the most common call types, listed in descending order by count?**

**5) What zip codes accounted for most common calls and what type were they?**

1. Filter out by Call Type
2. Group them by Call Type and Zipcode of Incident
3. Count them and display in descending order

**6) What neighbourhoods are in the two top zip codes from the listing in the previous question?**

Probably these two zip codes are somehow related to contested neighbourhood with high reported crimes.

# Additional exercise

Using the above dataset, write down code to anwser the following questions:

1. What was the sum of all calls, average, min and max of the response times for calls (from the moment call is received till response is acknowledged and unit is on route)?

    Hint: Use the functions sum(), avg(), min() and max()

2. How many distinct years of data is in the CSV file?

    Hint: Use the year() SQL Spark function off the timestamp of the Call Date column

3. What week of the year in 2017 had the most fire calls?

    Hint: Use the weekofyear() SQL Spark function off the timestamp of the Call Date column
    
Furthermore, create some visualizations to better understand the results obtained.

# References

* Learning Spark - Lightning-Fast Data Analytics, 2nd Ed. J. Damji, B. Wenig, T. Das, and D. Lee. O'Reilly, 2020
* Spark: The Definitive Guide - Big Data Processing Made Simple, 1st Ed. B. Chambers and M. Zaharia. O'Reilly, 2018
* https://spark.apache.org/docs/latest
* https://docs.python.org/3/


* https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3
